In [1]:
import sys
import os
import h5py
import importlib
import multiprocessing
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append('../../../')

import matplotlib.pyplot as plt
import numpy as np
from SpecWizard_Input import ReadData
from SpecWizard_BuildInput import Build_Input
from SpecWizard_ProjectData import SightLineProjection
from SpecWizard_ComputeOpticaldepth import ComputeOpticaldepth

This class defines the B-spline kernel of Monaghan and Lattanzio, as used in Gadget-2


In [4]:
import SpecWizard_SaveOpticaldepth
SpecWizard_SaveOpticaldepth = importlib.reload(SpecWizard_SaveOpticaldepth)
from SpecWizard_SaveOpticaldepth import OpticalDepth_IO

In [78]:
Wizard = buildinput.read_from_yml('Wizard.yml')

something
Warning! ProjectionAxes NOT found. Setting default value : ('simx', 'simy', 'simz')
Warning! x-axis NOT found. Setting default value : 0
Warning! y-axis NOT found. Setting default value : 1
Warning! z-axis NOT found. Setting default value : 2
Warning! ProjectionStart NOT found. Setting default value : (0.5, 0.5, 0)
Warning! x-position NOT found. Setting default value : 0.5
Warning! y-position NOT found. Setting default value : 0.5
Warning! z-position NOT found. Setting default value : 0
Warning! ProjectionLength NOT found. Setting default value : 1
Warning! SightLength NOT found. Setting default value : 1


In [13]:
import random as rd
rd.seed(3)
IC = np.array([[rd.randint(5,100)/100,rd.randint(5,100)/100] for i in range(500)])

In [16]:
zmin       = 0
zmax       = 6.1

In [83]:
snapdir = '/cosma7/data/Eagle/ScienceRuns/Planck1/L0050N0752/PE/REFERENCE/data/'
allfiles      = os.listdir(snapdir)

In [85]:
z_snaps = []
zs      = []
z_fnames   = []
for snap in allfiles:
    if snap.startswith("snapshot"):
        try: 
            z = float(snap[15:].replace("p","."))
            if (z >= zmin and z <= zmax):
                z_snaps.append(snap)
                zs.append(z)
                z_fnames.append(snap.replace("snapshot","snap")+".0.hdf5")
        except:
            pass
zs      = np.array(zs)
sorted_mask = np.argsort(zs.astype('float'))
z_snaps = np.array(z_snaps)[sorted_mask]
zs      = np.array(zs)[sorted_mask]
z_fnames = np.array(z_fnames)[sorted_mask]

In [97]:
for z_fname,z_snap in zip(z_fnames,z_snaps):
    
    print(z_snap,z_fname)
    Wizard['snapshot_params']['directory'] =snapdir + z_snap + '/'
    Wizard['snapshot_params']['file'] =  z_fname
    x_start,y_start = IC[0]
    Wizard['sightline']['ProjectionStart'] = (x_start,y_start,0)
    Wizard['sightline']['x-position'] = x_start
    Wizard['sightline']['y-position'] = y_start
    
    read_data = ReadData(Wizard)
    particles = read_data.read_particles()
    sightlineprojection  = SightLineProjection(Wizard)
    projected_LOS        = sightlineprojection.ProjectData(particles)
    cspec          = ComputeOpticaldepth(Wizard)
    opticaldepth   = cspec.MakeAllOpticaldepth(projected_LOS)

    def do_sight(isight, projections):

        x_start,y_start = IC[isight]
        Wizard['sightline']['ProjectionStart'] = (x_start,y_start,0)
        Wizard['sightline']['x-position'] = x_start
        Wizard['sightline']['y-position'] = y_start
        read_data = ReadData(Wizard)
        particles = read_data.read_particles()
        sightlineprojection  = SightLineProjection(Wizard)
        projected_LOS        = sightlineprojection.ProjectData(particles)
        cspec          = ComputeOpticaldepth(Wizard)
        opticaldepth   = cspec.MakeAllOpticaldepth(projected_LOS)

        # generate the result and store it in the dictionary
        result                = {"nsight": isight,
                                 'Projection':projected_LOS,
                                 'OpticaldepthWeighted':opticaldepth}
        projections[isight] = result

    ncpu   = 10      # number of cores to be used
    nsight = 250     # number of sight lines to compute
    #    
    pool   = multiprocessing.Pool(ncpu)
    manager      = multiprocessing.Manager()
    projections  = manager.dict()

    for i in range(nsight):
        pool.apply_async(do_sight, args=(i, projections))
    pool.close()
    pool.join()


    Wizard["Output"] = {}
    Wizard["Output"]["directory"] = './'
    Wizard["Output"]["fname"]     = 'spectra_L050N0750_'+z_snap+".hdf5"
    Wizard["snapshot_params"]['file'] = str(Wizard["snapshot_params"]['file'])
    SW_IO = OpticalDepth_IO(Wizard,True)
    for i in projections.keys():
        SW_IO.write_to_file(projections[i])


snapshot_028_z000p000 snap_028_z000p000.0.hdf5
Removing file  ./spectra_L050N0750_snapshot_028_z000p000.hdf5
snapshot_027_z000p101 snap_027_z000p101.0.hdf5
snapshot_026_z000p183 snap_026_z000p183.0.hdf5
snapshot_025_z000p271 snap_025_z000p271.0.hdf5
snapshot_024_z000p366 snap_024_z000p366.0.hdf5
snapshot_023_z000p503 snap_023_z000p503.0.hdf5
snapshot_022_z000p615 snap_022_z000p615.0.hdf5
snapshot_021_z000p736 snap_021_z000p736.0.hdf5
snapshot_020_z000p865 snap_020_z000p865.0.hdf5
snapshot_019_z001p004 snap_019_z001p004.0.hdf5
snapshot_018_z001p259 snap_018_z001p259.0.hdf5
snapshot_017_z001p487 snap_017_z001p487.0.hdf5
snapshot_016_z001p737 snap_016_z001p737.0.hdf5
snapshot_015_z002p012 snap_015_z002p012.0.hdf5
snapshot_014_z002p237 snap_014_z002p237.0.hdf5
snapshot_013_z002p478 snap_013_z002p478.0.hdf5
snapshot_012_z003p017 snap_012_z003p017.0.hdf5
snapshot_011_z003p528 snap_011_z003p528.0.hdf5
snapshot_010_z003p984 snap_010_z003p984.0.hdf5
snapshot_009_z004p485 snap_009_z004p485.0.hdf